In [0]:
!pip install tabulate


In [0]:
!pip install "colorama>=0.3.8"

In [0]:
!pip install future

In [0]:
!pip install requests

In [0]:
!pip install -f http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html h2o

Looking in links: http://h2o-release.s3.amazonaws.com/h2o/latest_stable_Py.html
     |████████████████████████████████| 122.7MB 292kB/s 
  Created wheel for h2o: filename=h2o-3.26.0.2-py2.py3-none-any.whl size=122725647 sha256=30163ef95a243874889ed80cadf4ba714c4543260bbaba36704d4eada3457897
  Stored in directory: /root/.cache/pip/wheels/76/01/62/d6b7bfc9d599463401118db95eab37c134a6f24040828d31ab
Successfully built h2o


In [0]:
import h2o
from h2o.automl import H2OAutoML

In [0]:
h2o.init()

# Import a sample binary outcome train/test set into H2O
train = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_train_10k.csv")
test = h2o.import_file("https://s3.amazonaws.com/erin-data/higgs/higgs_test_5k.csv")

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.4" 2019-07-16; OpenJDK Runtime Environment (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3); OpenJDK 64-Bit Server VM (build 11.0.4+11-post-Ubuntu-1ubuntu218.04.3, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6tc3rhej
  JVM stdout: /tmp/tmp6tc3rhej/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp6tc3rhej/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,27 days
H2O cluster name:,H2O_from_python_unknownUser_k40r9f
H2O cluster total nodes:,1
H2O cluster free memory:,2.938 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
# Identify predictors and response
x = train.columns
print(x)


['response', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28']


In [0]:
y = "response"
x.remove(y)
print(x)

['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28']


In [0]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
print(train[y])

response
1
1
1
0
1
0
1
1
1
1


In [0]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=20, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [0]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20190823_171019,0.789011,0.55215,0.327693,0.432579,0.187125
StackedEnsemble_BestOfFamily_AutoML_20190823_171019,0.787578,0.553086,0.311321,0.433157,0.187625
XGBoost_3_AutoML_20190823_171019,0.785002,0.556677,0.311308,0.43457,0.188851
XGBoost_3_AutoML_20190823_172506,0.785002,0.556677,0.311308,0.43457,0.188851
XGBoost_grid_1_AutoML_20190823_172506_model_3,0.784085,0.559335,0.326026,0.43543,0.189599
XGBoost_grid_1_AutoML_20190823_171019_model_3,0.784085,0.559335,0.326026,0.43543,0.189599
XGBoost_grid_1_AutoML_20190823_171019_model_1,0.783071,0.558937,0.324014,0.435565,0.189717
XGBoost_grid_1_AutoML_20190823_172506_model_1,0.783071,0.558937,0.324014,0.435565,0.189717
XGBoost_2_AutoML_20190823_172506,0.782426,0.557296,0.326804,0.435181,0.189383
XGBoost_2_AutoML_20190823_171019,0.782426,0.557296,0.326804,0.435181,0.189383


In [0]:
# The leader model is stored here
aml.leader

# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

# or:
preds = aml.leader.predict(test)
print(preds)

stackedensemble prediction progress: |████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1
0,0.741147,0.258853
0,0.688509,0.311491
1,0.568763,0.431237
1,0.257548,0.742452
0,0.691581,0.308419
1,0.318009,0.681991
1,0.297098,0.702902
0,0.662363,0.337637
1,0.597819,0.402181
0,0.829927,0.170073
